In [1]:
import os
import warnings
os.chdir('../..')
warnings.filterwarnings('ignore')
# Firstly import the class of dataset
from Scripts.Data_Loader import EIRDataset

EIR_Dataset = EIRDataset('./Generated/Data_Train/', task_type='geometric', n_jobs=72) # task type can be `geometric` or `random` or `all`

Loading .fif files: 100%|██████████| 434/434 [00:14<00:00, 29.19it/s]


In [2]:
def resample_df(EIR_Dataset: EIRDataset, freq: int):
    for i in range(len(EIR_Dataset)): 
        eeg_sample, eye_sample, metadata, label, img = EIR_Dataset[i]
        eeg_sample.resample(freq)
resample_df(EIR_Dataset, 256)

In [3]:
import numpy as np
loaded = np.load('./Generated/Spectrums/exec_morlets.npz')

results_arr = []
i = 0
while f'power_{i}' in loaded:
    power = loaded[f'power_{i}']
    phase = loaded[f'phase_{i}']
    s_id = int(loaded[f'subject_id_{i}'])
    t_id = int(loaded[f'trial_id_{i}'])
    gender = str(loaded[f'gender_{i}'])
    handiness = str(loaded[f'handiness_{i}'])
    age = int(loaded[f'age_{i}'])
    label = int(loaded[f'label_{i}'])
    img = loaded[f'img_{i}']
    task_type = str(loaded[f'task_type_{i}'])
    
    results_arr.append([power, phase, s_id, t_id, gender, handiness, age, label, img, task_type])
    i += 1

power, phase, s_id, t_id, gender, handiness, age, label, img, task_type = results_arr[0]


In [4]:
power.shape

(63, 195, 321)

In [5]:
subjects_cluster3 = [7, 7, 9, 9, 14]
trial_cluster3 = [2, 1, 2, 1, 1]
subjects_cluster4 = [5, 5, 3, 3, 2, 2, 10, 10, 12, 12, 16, 16, 11, 11, 13, 13]
trial_cluster4 = [2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
selected_trials = trial_cluster3+trial_cluster4
selected_subjects= subjects_cluster3+subjects_cluster4
power_wav = []
phase_wav = []
subj = []
trial = []
label_wav = []
tasks = []

max_len = 309

for i, sample in enumerate(results_arr):

    if i == len(EIR_Dataset):
        break
    # for j in range(len(selected_trials)):
        # if i[9] == 'g' and i[2] == selected_subjects[j] and i[3] == selected_trials[j]:
    power_wav.append(sample[0][:, :, :max_len])
    phase_wav.append(sample[1][:, :, :max_len])
    subj.append(sample[2])
    trial.append(sample[3])
    label_wav.append(sample[7])
    
power_wav = np.array(power_wav)
phase_wav = np.array(phase_wav)

In [6]:
power_wav.shape

(434, 63, 195, 309)

In [7]:
phase_wav.shape

(434, 63, 195, 309)

In [8]:
power_wav_combined = power_wav.transpose(0, 2, 1, 3).reshape(power_wav.shape[0], power_wav.shape[1], power_wav.shape[2]*power_wav.shape[3])
phase_wav_combined = phase_wav.transpose(0, 2, 1, 3).reshape(phase_wav.shape[0], phase_wav.shape[1], phase_wav.shape[2]*phase_wav.shape[3])

In [9]:
power_wav_combined.shape

(434, 63, 60255)

In [ ]:
from sklearn.model_selection import train_test_split
from Scripts import Selectors_From_Dataset as sel
X, img, y = sel.get_sample(EIR_Dataset)

def take_subset(n=20, random_state=42, *args):
    rng = np.random.default_rng(random_state)
    N = len(y)
    n = min(n, N)
    idx = rng.choice(N, size=n, replace=False)

    returnable = []
    for value in args:
        returnable.append(value[idx])

    return returnable

# X, y, power_wav_combined, phase_wav_combined, label_wav = take_subset(20, 123, X, y, power_wav_combined, phase_wav_combined, np.array(label_wav))
# subjects_cluster3 = [7, 7, 9, 9, 14]
# trial_cluster3 = [2, 1, 2, 1, 1]
# subjects_cluster4 = [5, 5, 3, 3, 2, 2, 10, 10, 12, 12, 16, 16, 11, 11, 13, 13]
# trial_cluster4 = [2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
# X, img, y = sel.get_sample_choosen_trial(EIR_Dataset, subjects_cluster4 + subjects_cluster3, trial_cluster4 + trial_cluster3)

# X_train, X_test, Y_train, Y_test, power_wav_train, power_wav_test, phase_wav_train, phase_wav_test, y_wav_train, y_wav_test = train_test_split(
#     X, y, power_wav_combined, phase_wav_combined, np.array(label_wav), test_size=0.5, stratify=y, random_state=40, 
# )

In [11]:
X.shape

(20, 63, 4096)

In [12]:
import numpy as np
import scipy
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
import matplotlib.pyplot as plt
from pyriemann.estimation import Covariances
from typing import Union

n_components = 4
class ModelCovariance:
    class ParallelCovariances(BaseEstimator, TransformerMixin):
        def __init__(self, n_filters, covariance_type: Union[list, type] = Covariances):
            self.covariance_type = covariance_type
            self.n_filters = n_filters
            self.cov = []
            self.cov_pipeline = []

            for i, cov_type in enumerate(covariance_type):
                if cov_type == XdawnCovariances:
                    self.cov.append(cov_type(nfilter = self.n_filters[i], estimator = 'lwf', xdawn_estimator='lwf'))
                else: self.cov.append(cov_type(estimator = 'lwf'))
        def fit(self, X, y):
            self.cov_pipeline = []
            for i in range(len(X)):
                self.cov_pipeline.append(Pipeline([
                ('covariance', self.cov[i]),
                ('tangent', TangentSpace())]))
                self.cov_pipeline[i].fit(X[i], y)
            return self
        
        def transform(self, X):
            ret = []
            for i in range(len(X)):
                ret.append(self.cov_pipeline[i].transform(X[i]))
            combined = np.concatenate(ret, axis=1)
            return combined

        
    def __init__(self, feature_groups: int = 3, n_filters: Union[list, int, None] = 4, covariances: Union[list, type] = XdawnCovariances,
                  Classifier: Union[list, BaseEstimator] = LogisticRegression(C=3, class_weight='balanced', max_iter = 300)):
        self.feature_groups = feature_groups
        self.covariances = covariances
        self.n_filters = n_filters
        if feature_groups == 1: # Предполагается что в этом случае нигде не используется list для передачи параметров
            cov = covariances(estimator='lwf')
            if covariances == XdawnCovariances:
                cov = covariances(n_filters, estimator='lwf', xdawn_estimator='lwf')
            self.pipeline = Pipeline([
                ('cov', cov),
                ('tang', TangentSpace()),
                ('clf', Classifier)])
        else: # Если используем листы для передачи параметров - следим чтобы длина совпадала с feature_groups
            if isinstance(n_filters, int):
                self.n_filters = [n_filters] * feature_groups
            if isinstance(covariances, type):
                self.covariances = [covariances] * feature_groups
            self.pipeline = (Pipeline([
                ('cov', self.ParallelCovariances(self.n_filters, self.covariances)),
                ('clf', Classifier)]))
    def get_pipeline(self):
        return self.pipeline
    def fit(self, X, y):
        self.pipeline.fit(X, y)
        return self

    def predict(self, X):
        return self.pipeline.predict(X)
    
    def evaluate(self, X, true_y, metric: Union[str, list] = 'accuracy'):
        preds = self.pipeline.predict(X)
        if isinstance(metric, str):
            if metric == 'accuracy':
                return metrics.accuracy_score(true_y, preds)
            elif metric == 'precision':
                return metrics.precision_score(true_y, preds, average='weighted')
            elif metric == 'recall':
                return metrics.recall_score(true_y, preds, average='weighted')
            elif metric == 'f1':
                return metrics.f1_score(true_y, preds, average='weighted')
            elif metric == 'roc_auc':
                return metrics.roc_auc_score(true_y, preds)
            else:
                raise ValueError(f"Unknown metric: {metric}")
        
        elif isinstance(metric, list):
            results = {}
            for m in metric:
                if m == 'accuracy':
                    results[m] = metrics.accuracy_score(true_y, preds)
                elif m == 'precision':
                    results[m] = metrics.precision_score(true_y, preds, average='weighted')
                elif m == 'recall':
                    results[m] = metrics.recall_score(true_y, preds, average='weighted')
                elif m == 'f1':
                    results[m] = metrics.f1_score(true_y, preds, average='weighted')
                elif m == 'roc_auc':
                    results[m] = metrics.roc_auc_score(true_y, preds)
                else:
                    raise ValueError(f"Unknown metric: {m}")
            return results

###  Train-Test Split

In [13]:
# import numpy as np
# from itertools import product
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from tqdm import tqdm
# from sklearn import metrics

# # Вставь сюда свои данные: X, y, power_wav_combined, phase_wav_combined, label_wav
# # from your_data_module import X, y, power_wav_combined, phase_wav_combined, label_wav

# random_states = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
# test_sizes = [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.7]

# # Словари для хранения метрик
# results_storage = {
#     1: [],
#     2: [],
#     3: [],
#     4: [],
#     5: []
# }

# # Основной цикл по всем комбинациям
# for random_state, test_size in tqdm(list(product(random_states, test_sizes))):
#     try:
#         # Сплитим данные
#         X_train, X_test, Y_train, Y_test, power_wav_train, power_wav_test, phase_wav_train, phase_wav_test, y_wav_train, y_wav_test = train_test_split(
#             X, y, power_wav_combined, phase_wav_combined, np.array(label_wav),
#             test_size=test_size, stratify=y, random_state=random_state
#         )
    
#         # Определяем модели
#         models = {
#             1: ModelCovariance(feature_groups=1, n_filters=4,
#                                covariances=XdawnCovariances,
#                                Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
#             2: ModelCovariance(feature_groups=3, n_filters=4,
#                                covariances=[XdawnCovariances, Covariances, Covariances],
#                                Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
#             3: ModelCovariance(feature_groups=2, n_filters=4,
#                                covariances=[XdawnCovariances, Covariances],
#                                Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
#             4: ModelCovariance(feature_groups=2, n_filters=4,
#                                covariances=[XdawnCovariances, Covariances],
#                                Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
#             5: ModelCovariance(feature_groups=2, n_filters=4,
#                                covariances=[Covariances, Covariances],
#                                Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
#         }
    
#         inputs_train = {
#             1: X_train,
#             2: [X_train, power_wav_train, phase_wav_train],
#             3: [X_train, power_wav_train],
#             4: [X_train, phase_wav_train],
#             5: [power_wav_train, phase_wav_train]
#         }
    
#         inputs_test = {
#             1: X_test,
#             2: [X_test, power_wav_test, phase_wav_test],
#             3: [X_test, power_wav_test],
#             4: [X_test, phase_wav_test],
#             5: [power_wav_test, phase_wav_test]
#         }
    
#         # Обучение и оценка моделей
#         for i in range(1, 6):
#             models[i].fit(inputs_train[i], Y_train)
#             results = models[i].evaluate(inputs_test[i], Y_test, ['accuracy', 'f1'])
#             results_storage[i].append({
#                 'random_state': random_state,
#                 'test_size': test_size,
#                 'accuracy': results['accuracy'],
#                 'f1': results['f1']
#             })
#     except Exception as e:
#         print(f"ERROR AT: random_state={random_state}, test_size={test_size}.\nERROR:{e}")

### K-fold

In [15]:
import numpy as np
from itertools import product
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn import metrics

random_states = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
n_splits = 3

results_storage = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: []
}

metrics_list = ['accuracy', 'f1']

models = {
                1: ModelCovariance(feature_groups=1, n_filters=4,
                                    covariances=XdawnCovariances,
                                    Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
                2: ModelCovariance(feature_groups=3, n_filters=4,
                                    covariances=[XdawnCovariances, Covariances, Covariances],
                                    Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
                3: ModelCovariance(feature_groups=2, n_filters=4,
                                    covariances=[XdawnCovariances, Covariances],
                                    Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
                4: ModelCovariance(feature_groups=2, n_filters=4,
                                    covariances=[XdawnCovariances, Covariances],
                                    Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
                5: ModelCovariance(feature_groups=2, n_filters=4,
                                    covariances=[Covariances, Covariances],
                                    Classifier=LogisticRegression(C=3, class_weight='balanced', max_iter=300)),
            }

# Основной цикл — теперь по random_state (каждый задаёт разное разбиение фолдов)
for random_state in tqdm(random_states):
    try:
        k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

        for train_index, test_index in k_fold.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = y[train_index], y[test_index]
            power_wav_train, power_wav_test = power_wav_combined[train_index], power_wav_combined[test_index]
            phase_wav_train, phase_wav_test = phase_wav_combined[train_index], phase_wav_combined[test_index]
            y_wav_train, y_wav_test = np.array(label_wav)[train_index], np.array(label_wav)[test_index]

            inputs_train = {
                1: X_train,
                2: [X_train, power_wav_train, phase_wav_train],
                3: [X_train, power_wav_train],
                4: [X_train, phase_wav_train],
                5: [power_wav_train, phase_wav_train]
            }

            inputs_test = {
                1: X_test,
                2: [X_test, power_wav_test, phase_wav_test],
                3: [X_test, power_wav_test],
                4: [X_test, phase_wav_test],
                5: [power_wav_test, phase_wav_test]
            }

            # Обучение и оценка моделей
            for i in range(1, 6):
                models[i].fit(inputs_train[i], Y_train)
                results = models[i].evaluate(inputs_test[i], Y_test, metrics_list)
                
                storage_item = {
                    'random_state': random_state,
                    'fold': len(results_storage[i]) % n_splits + 1}
                for metric in metrics_list:
                    storage_item[metric] = results[metric]
                    
                results_storage[i].append(storage_item)

    except Exception as e:
        print(f"ERROR AT: random_state={random_state}.\nERROR: {e}")

100%|██████████| 11/11 [27:24<00:00, 149.49s/it]


### Remaining code

In [19]:
import plotly.graph_objects as go
import numpy as np

metric = 'f1'
if metric not in metrics_list:
    print(f"ERROR: {metric} is not in metrics_list and has not been evaluzted")

# Выбираем эксперимент
for i in range(1, 6):
    exp = i  # от 1 до 5
    
    # Извлекаем и сортируем данные
    f1_data = results_storage[exp]
    
    # Получаем уникальные значения по осям
    random_states_unique = sorted(set(m['random_state'] for m in f1_data))
    test_sizes_unique = sorted(set(m['fold'] for m in f1_data))
    
    # Инициализируем матрицу f1_scores (ось Y — test_size, ось X — random_state)
    Z = np.zeros((len(test_sizes_unique), len(random_states_unique)))
    
    # Заполняем матрицу Z
    for m in f1_data:
        i = test_sizes_unique.index(m['fold'])
        j = random_states_unique.index(m['random_state'])
        Z[i, j] = m[metric]
    
    # Построение 3D поверхности
    fig = go.Figure(data=[
        go.Surface(
            z=Z,
            x=random_states_unique,
            y=test_sizes_unique,
            colorscale='Viridis',
            colorbar=dict(title=f'{metric} Score'),
            showscale=True
        )
    ])
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Random State',
            yaxis_title='Test Size',
            zaxis_title=f'{metric} Score'
        ),
        title=f'{metric} Score Surface for Experiment {exp}',
        margin=dict(l=0, r=0, b=0, t=40)
    )
    
    fig.write_html(f"f1_xDAWN_LR_Exp{exp}_(ALL_SAMPLES).html")
    
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [20]:
from bs4 import BeautifulSoup
import json
import numpy as np
import matplotlib.pyplot as plt
import re
import base64
import os

# Пути к HTML-файлам
base_path = "./Generated/Figures/Classification"
file_names = [
    f"{metric}_xDAWN_LR_Exp1_(ALL_SAMPLES).html",
    f"{metric}_xDAWN_LR_Exp2_(ALL_SAMPLES).html",
    f"{metric}_xDAWN_LR_Exp3_(ALL_SAMPLES).html",
    f"{metric}_xDAWN_LR_Exp4_(ALL_SAMPLES).html",
    f"{metric}_xDAWN_LR_Exp5_(ALL_SAMPLES).html"
]

# Названия для легенды
labels = ['EEG',
          'EEG + Power + Phase',
          'EEG + Power',
          'EEG + Phase',
          'Power + Phase']

# Списки для хранения средних и std
means = []
stds = []

plt.figure(figsize=(10, 6))

# Обработка каждого эксперимента
for file_name, label in zip(file_names, labels):
    # 1. Считать HTML
    with open(os.path.join(base_path, file_name), encoding="utf-8") as f:
        html = f.read()

    # 2. Найти Plotly JSON
    pattern = re.compile(r'Plotly\.newPlot\([^,]+,\s*(\[\{.*?\}\])', re.DOTALL)
    match = pattern.search(html)
    if not match:
        print(f"❌ Не найден Plotly JSON в файле {file_name}")
        continue

    plot_data_json = match.group(1)
    plot_data = json.loads(plot_data_json)[0]

    x = plot_data['x']
    y = plot_data['y']
    z_meta = plot_data['z']

    # 3. Распаковать z
    z_bytes = base64.b64decode(z_meta['bdata'])
    z_array = np.frombuffer(z_bytes, dtype=z_meta['dtype'])
    z = z_array.reshape(tuple(map(int, z_meta['shape'].split(','))))

    # 4. Среднее по X (axis=1)
    z_avg_by_y = z.mean(axis=1)

    # 5. Нарисовать
    plt.plot(y, z_avg_by_y, marker='o', label=label)

    # 6. Посчитать общее среднее и std по всей матрице z
    z_mean = z.mean()
    z_std = z.std()
    means.append(z_mean)
    stds.append(z_std)

# Оформление графика
plt.xlabel("Test Size")
plt.ylabel(f"Average {metric} Score")
plt.title(f"{metric} Score vs Test Size")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Печать результатов
print(f"\n📊 Mean {metric} scores:")
for label, mean in zip(labels, means):
    print(f"{label}: {mean:.4f}")

print(f"\n📈 Std {metric} scores:")
for label, std in zip(labels, stds):
    print(f"{label}: {std:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: './Generated/Figures/Classification/f1_xDAWN_LR_Exp1_(ALL_SAMPLES).html'

<Figure size 1000x600 with 0 Axes>